## Experiments for Trustworthy AI Project

Code is pretty rough.. 
If you don't want to run the experiment again, please just load `reclor_1000_response.json` file

#### 1. Installing Packages and Dependencies 

In [1]:
# make sure that you installed those dependencies; ChatGPT won't work without the latest version of openai
%pip install openai
%pip install python-dotenv
%pip install requests
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
# import the necessary libraries
import os
import json
from decouple import config

import matplotlib.pyplot as plt
import pandas as pd
import openai
import random

In [15]:
from decouple import config

OPENAI_API_KEY = config('API_KEY')
openai.api_key = OPENAI_API_KEY


#### 2. Loading the Data 

In [12]:
# Load the JSON file from the reclor folder 
with open('./reclor_data/train.json') as f:
    data = json.load(f)

# Check the length of the data
print(len(data))

4638


In [39]:
# Load the JSON file
with open('./reclor_data/train.json') as f:
    reclor_train_data = json.load(f)

# Cut the data to 10, 50, 100, 1000 samples
reclor_train_data_10 = random.sample(reclor_train_data, 10)
reclor_train_data_50 = random.sample(reclor_train_data, 50)
reclor_train_data_100 = random.sample(reclor_train_data, 100)
reclor_train_data_1000 = random.sample(reclor_train_data, 1000)

#### 3.1 Experiment on ChatGPT

In [46]:
# begin the chatgpt experiment by prompting the model with a question in the sampled data

answer_list = []

for reclor_quesiton in reclor_train_data_1000:
    single_answer = {}
    context = reclor_quesiton['context']
    question = reclor_quesiton['question']
    answers = reclor_quesiton['answers']
    id_string = reclor_quesiton['id_string']
    label = reclor_quesiton['label']
    
    answer_obj = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a logician and an LSAT taker."},
            {"role": "user", "content": f'Given the context: {context} and question {question}. Please choose the correct answer among a list of answers given: {answers}. Give your the answer as a number, where 0 refers to the first answer in the answer list, 1 is the second answer in the answer list, 2 is the third answer in the answer list, and 3 is the fourth answer in the answer list. State this number first, then state your confidence in that answer, finally state the answer you have chosen. Sample Response "The answer is 0. I am 33 percent confident that my answer is right. People should put more trust in their government"'},
        ]
    )
    
    chatgpt_answer = answer_obj["choices"][0]["message"]["content"]
    
    single_answer['id'] = id_string
    single_answer['correct_answer'] = label
    single_answer['chat_gptanswer'] = chatgpt_answer
    
    answer_list.append(single_answer)
    
print(answer_list)


[{'id': 'train_3350', 'correct_answer': 2, 'chat_gptanswer': "\n\nThe answer is 2. I am 90% confident with my answer. The principle suggests that the Thimble Corporation has an obligation to rectify any unfair result of the situation they have created. Therefore, if there's a possibility that some customers did not apply for the rebate due to the incorrect expiration date, the Thimble Corporation should identify them and offer them the rebate to rectify the unfair result of their mistake."}, {'id': 'train_2036', 'correct_answer': 0, 'chat_gptanswer': '\n\nThe answer is 0. I am 90% confident that my answer is correct. The argument in the answer 0 shares the same flawed pattern of reasoning as the argument given in the context. Both arguments make assumptions about the relationship between a set and its subsets based on the presence or absence of a certain property in some elements of the set.'}, {'id': 'train_3344', 'correct_answer': 3, 'chat_gptanswer': 'The answer is 2. I am 90% confi

In [47]:
# Writ the answer in a JSON format, very important if you don't want to waste money. If you don't do this, I will be charged for every single request you make to the API
with open('reclor_1000_response.json', 'w') as file:
    json.dump(answer_list, file)


In [ ]:
import json
# if you want to load the JSON file, you can do it like this

with open('reclor_1000_response.json', 'r') as f:
    # Load the contents of the file into a list
    answer_list = json.load(f)

# Print the list
print(answer_list)


In [58]:
import re
import copy

# deep copy the list
answer_list_copy = copy.deepcopy(answer_list)

new_list1 = []
# Iterate through the list of dictionaries and update the 'chat_gptanswer' value
for item in answer_list_copy:
    # Use regular expressions to extract the answer label 
    response = {}
    
    answer_match = re.search(r'(\b[0-9]\b)', item['chat_gptanswer'])
    percentage_matches = re.findall(
        r'\b[1-9]\d{1,2}\b', item['chat_gptanswer'])

    response['id'] = item['id']
    response['correct_answer'] = item['correct_answer']
    if answer_match:
        # Update the dictionary value with the extracted number
        response['chat_gptanswer'] = int(answer_match.group(1))
    else:
        # If the regular expression doesn't match, set the value to None
        print(f'warning + {response["id"]}')
        continue  
    if percentage_matches: 
        response['confidence'] = int(percentage_matches[0])
    else: 
        print(f'warning + {response["id"]}')
        continue
    new_list1.append(response)
# Print the updated list of dictionaries
print(new_list1)


warning + train_4569
warning + train_4134
warning + train_1654
warning + train_3263
[{'id': 'train_3350', 'correct_answer': 2, 'chat_gptanswer': 2, 'confidence': 90}, {'id': 'train_2036', 'correct_answer': 0, 'chat_gptanswer': 0, 'confidence': 90}, {'id': 'train_3344', 'correct_answer': 3, 'chat_gptanswer': 2, 'confidence': 90}, {'id': 'train_4476', 'correct_answer': 0, 'chat_gptanswer': 1, 'confidence': 80}, {'id': 'train_4317', 'correct_answer': 1, 'chat_gptanswer': 1, 'confidence': 100}, {'id': 'train_3047', 'correct_answer': 3, 'chat_gptanswer': 2, 'confidence': 95}, {'id': 'train_136', 'correct_answer': 2, 'chat_gptanswer': 3, 'confidence': 100}, {'id': 'train_2884', 'correct_answer': 0, 'chat_gptanswer': 0, 'confidence': 90}, {'id': 'train_1896', 'correct_answer': 2, 'chat_gptanswer': 2, 'confidence': 90}, {'id': 'train_4594', 'correct_answer': 0, 'chat_gptanswer': 0, 'confidence': 90}, {'id': 'train_2172', 'correct_answer': 2, 'chat_gptanswer': 2, 'confidence': 80}, {'id': 'trai

In [65]:
import pandas as pd

df = pd.DataFrame(new_list1)
df.head()

,id,correct_answer,chat_gptanswer,confidence
0,train_3350,2,2,90
1,train_2036,0,0,90
2,train_3344,3,2,90
3,train_4476,0,1,80
4,train_4317,1,1,100


In [66]:
df['correct'] = df.apply(
    lambda row: 1 if row['correct_answer'] == row['chat_gptanswer'] else 0, axis=1)

percent_correct = df['correct'].sum() / len(df) * 100.0


In [67]:
df

,id,correct_answer,chat_gptanswer,confidence,correct
0,train_3350,2,2,90,1
1,train_2036,0,0,90,1
2,train_3344,3,2,90,0
3,train_4476,0,1,80,0
4,train_4317,1,1,100,1
...,...,...,...,...,...
991,train_4606,1,1,95,1
992,train_549,3,0,90,0
993,train_2458,1,1,90,1
994,train_4214,1,1,95,1


In [68]:
# calculate how many questions model got correct

percent_correct

58.23293172690763

In [69]:
df['confidence'].mean()

89.68975903614458

In [70]:
filtered_df = df[df['correct'] != 1]


In [72]:
filtered_df['confidence'].mean()

89.14663461538461

In [73]:
filtered_df = df[df['correct'] != 0]

In [74]:
filtered_df['confidence'].mean()

90.07931034482759

In [75]:
correct_df = df[df['correct_answer'] == df['chat_gptanswer']]

In [76]:
correct_df

,id,correct_answer,chat_gptanswer,confidence,correct
0,train_3350,2,2,90,1
1,train_2036,0,0,90,1
4,train_4317,1,1,100,1
7,train_2884,0,0,90,1
8,train_1896,2,2,90,1
...,...,...,...,...,...
988,train_4252,3,3,80,1
989,train_1388,3,3,90,1
991,train_4606,1,1,95,1
993,train_2458,1,1,90,1


In [77]:
# change the below to a list 
correct_df['id']

correct_lst = correct_df['id'].tolist()

print(correct_lst)

['train_3350', 'train_2036', 'train_4317', 'train_2884', 'train_1896', 'train_4594', 'train_2172', 'train_1272', 'train_1833', 'train_1655', 'train_1643', 'train_3951', 'train_79', 'train_2059', 'train_3964', 'train_2505', 'train_1828', 'train_746', 'train_3297', 'train_1017', 'train_1204', 'train_70', 'train_2133', 'train_4208', 'train_2602', 'train_3476', 'train_459', 'train_4527', 'train_4017', 'train_3320', 'train_3669', 'train_4362', 'train_714', 'train_2653', 'train_1366', 'train_3778', 'train_4193', 'train_3514', 'train_1601', 'train_2442', 'train_2813', 'train_3303', 'train_3774', 'train_1893', 'train_2184', 'train_3298', 'train_4344', 'train_4365', 'train_635', 'train_3841', 'train_2820', 'train_1436', 'train_1656', 'train_772', 'train_673', 'train_1679', 'train_2168', 'train_3800', 'train_3009', 'train_166', 'train_4218', 'train_4486', 'train_2004', 'train_4323', 'train_995', 'train_788', 'train_2334', 'train_1475', 'train_3995', 'train_4364', 'train_2556', 'train_4316', 'tra

In [79]:
list_of_correct_gpt3=_answers = []

for item in reclor_train_data_1000:
    if item['id_string'] in correct_lst:
        list_of_correct_gpt3.append(item)


In [80]:
len(list_of_correct_gpt3)


580

#### 3.2 Experiment on GPT3

In [105]:
gpt_answer_list = []

for reclor_quesiton in reclor_train_data_1000:
    single_answer = {}
    context = reclor_quesiton['context']
    quesiton = reclor_quesiton['question']
    answers = reclor_quesiton['answers']
    id_string = reclor_quesiton['id_string']
    label = reclor_quesiton['label']

    answer_obj = openai.Completion.create(
        model="text-davinci-003",
        max_tokens=300,
        prompt=f'Context: {context} and question {quesiton} choosing the correct answer among a list of answers in {answers}. State the answer you have chosen, then state its position in the list of answers. For example, 0 if it is the first answer; 1 if it is the second one; 2 if it is the third one, 3 if it is the fourth one. Give me the response in this format: "The answer label is 1. Not all people are bad at math." Answer: ?',
    )

    gpt3_answer = answer_obj["choices"][0]["text"]

    single_answer['id'] = id_string
    single_answer['correct_answer'] = label
    single_answer['gpt3_answer'] = gpt3_answer

    gpt_answer_list.append(single_answer)

print(gpt_answer_list)


[{'id': 'train_3350', 'correct_answer': 2, 'gpt3_answer': ' The answer is: "The answer is 3. If there is a chance that any customers did not apply for the rebate because of an incorrect expiration date on their rebate coupon, the Thimble Corporation is obligated to try to identify them and offer them the rebate." The answer label is 2.'}, {'id': 'train_2036', 'correct_answer': 0, 'gpt3_answer': '\n\nThe answer is 3. Not all teachers have good communications skills, but some teachers have good writing skills. Ttherefore, all teachers have either good communications skills or good writing skills. This is the fourth answer in the list.'}, {'id': 'train_3344', 'correct_answer': 3, 'gpt3_answer': 'The answer is No gold coins were minted that had a gold content of less than 92 percent. and the answer label is 0.'}, {'id': 'train_4476', 'correct_answer': 0, 'gpt3_answer': "The answer label is 1. 'The first-graders who wrote letters the most automatically before receiving the after-school less

In [100]:
gpt_answer_list


[{'id': 'train_2980',
  'correct_answer': 0,
  'gpt3_answer': ' The answer label is 0. People who engage in an activity and have a financial stake in that activity should not be the sole regulators of that activity.'},
 {'id': 'train_1886',
  'correct_answer': 2,
  'gpt3_answer': ' The answer is 3. "This year no more than half of the members of the club\'s committee that makes decisions about applicants\' qualifications were men."'},
 {'id': 'train_1222',
  'correct_answer': 1,
  'gpt3_answer': ' The answer chosen is "desiring to conform to a principle requires free will". The position of this answer in the list of answers is 1.'},
 {'id': 'train_1769',
  'correct_answer': 3,
  'gpt3_answer': 'The answer label is 3. The plastics used by the company in its products were, after a recent investigation by a regulatory body, declared to be safe for consumers.'},
 {'id': 'train_1452',
  'correct_answer': 0,
  'gpt3_answer': ' The answer label is 3. Modern technology can make small cars more 

In [107]:
with open('reclor_1000_gpt3_response.json', 'w') as file:
    json.dump(gpt_answer_list, file)


In [ ]:
# if you just want to load the file and not run the code above

import json

# Open the JSON file
with open('reclor_1000_gpt3_response.json', 'r') as f:
    # Load the contents of the file into a list
    gpt_answer_list = json.load(f)

# Print the list
print(gpt_answer_list)


In [114]:
import re
import copy

# Your original list of dictionaries
# deep copy the list
copy_gpt_list = copy.deepcopy(gpt_answer_list)
final_gpt_list = []

# Iterate through the list of dictionaries and update the 'chat_gptanswer' value
for item in gpt_answer_list:
    # Use regular expressions to extract the answer label
    response = {}
    response['id'] = item['id']
    response['correct_answer'] = item['correct_answer']
    
    # Create a regular expression pattern to match the answer label
    regex_pattern = re.compile(r'\b(first|second|third|fourth)\b')
    item['gpt3_answer'] = regex_pattern.sub(lambda m: {
                                            'first': '0', 'second': '1', 'third': '2', 'fourth': '3'}[m.group(1)], item['gpt3_answer'])

    answer_match = re.search(r'(\b[0-9]\b)', item['gpt3_answer'])

    if answer_match:
        # Update the dictionary value with the extracted number
        response['gpt3_answer'] = int(answer_match.group(1))
    else:
        # If the regular expression doesn't match, set the value to None
        print(item['id'])
        continue
    
    final_gpt_list.append(response)
    


train_4208
train_2149
train_4176
train_3628
train_1795
train_325


In [115]:
import pandas as pd

df = pd.DataFrame(final_gpt_list)
df.head()

df['percent_correct'] = df.apply(
    lambda row: 1.0 if row['correct_answer'] == row['gpt3_answer'] else 0.0, axis=1)

percent_correct = df['percent_correct'].sum() / len(df) * 100.0

print(percent_correct)

47.68611670020121
